# **Data Analysis**
**With Spark SQL & R Programming Language and Power BI report**

This Project aim to Explore and Develop a BI Solution for *2022 Report* Using Microsoft Fabric - Azure Data Engineering.

##### Reading Data in Lake house with SQL Query

In [1]:
SELECT COUNT(*) AS TotalRecords  
FROM Hackathon_LH.dbo_UnadjustedEmissions;


StatementMeta(, ae82ef6c-98e1-4d94-be44-0bbf7e6924c1, 2, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

##### Let get familiar with the dataset and limiting query results to 5row for faster responses.

In [22]:
SELECT  * 
FROM Hackathon_LH.dbo_UnadjustedEmissions 
Where UnadjustedNOxEmissions > 0
LIMIT 5


StatementMeta(, fe781bfa-1b97-4495-9ed2-5f73db71e60e, 24, Finished, Available)

<Spark SQL result set with 5 rows and 23 fields>

In [11]:
SELECT  * FROM Hackathon_LH.dbo_UnadjustedEmissions LIMIT 5

StatementMeta(, c561cafb-184d-4bae-8a49-40e6b08b4895, 12, Finished, Available)

<Spark SQL result set with 5 rows and 23 fields>

## **List of power plants with lower emission output than the average input**
##### This means that the power plant's are efficiently in converting fuel into electricity while minimizing emissions.

In [18]:
SELECT DISTINCT FacilityCode, PlantName, EmissionOutput,EmissionIutput, UtilityName,StateABB
FROM Hackathon_LH.dbo_UnadjustedEmissions
WHERE EmissionOutput < (SELECT AVG(EmissionIutput) FROM Hackathon_LH.dbo_UnadjustedEmissions)
ORDER BY EmissionOutput DESC
LIMIT 5
 

StatementMeta(, c561cafb-184d-4bae-8a49-40e6b08b4895, 19, Finished, Available)

<Spark SQL result set with 5 rows and 6 fields>

## **List of Power plants with higher emission output than average input rate lb/MWh.**
##### When a NOx Emission output rate is greater than the input rate, It means the power plant process more emit pollutants than the energy it produces.

In [37]:
--This query returns the top 5 power plants that emit more NOx pollution than their average electricity input.
SELECT DISTINCT FacilityCode, StateABB,   EmissionOutput, PlantName, UtilityName
FROM Hackathon_LH.dbo_UnadjustedEmissions
WHERE EmissionOutput > (SELECT AVG(EmissionIutput) FROM Hackathon_LH.dbo_UnadjustedEmissions)
    AND YEAR = 2022
 ORDER BY EmissionOutput DESC
LIMIT 5;


StatementMeta(, c561cafb-184d-4bae-8a49-40e6b08b4895, 38, Finished, Available)

<Spark SQL result set with 5 rows and 5 fields>

In [42]:
--This query returns the top 5 power plants that emit more NOx pollution than their average electricity input.
--**This query is similar to the query above. ( I am adding an aggregation function to the select statement with Group By )**
SELECT FacilityCode, StateABB, MAX(EmissionOutput) AS MaxEmissionOutput, PlantName, UtilityName
FROM Hackathon_LH.dbo_UnadjustedEmissions
WHERE EmissionOutput > (SELECT AVG(EmissionIutput) FROM Hackathon_LH.dbo_UnadjustedEmissions)
    AND YEAR = 2022
GROUP BY FacilityCode, StateABB, PlantName, UtilityName
ORDER BY MaxEmissionOutput DESC
LIMIT 5;


StatementMeta(, c561cafb-184d-4bae-8a49-40e6b08b4895, 43, Finished, Available)

<Spark SQL result set with 5 rows and 5 fields>

In [43]:
--This query returns the top power plants that emit more NOx pollution than their average electricity input in 2022  in Tennessee, Arkansas, and Mississippi.
WITH MaxEmissionPerState AS (
    SELECT 
        FacilityCode, 
        StateABB, 
        EmissionOutput,
        MAX(EmissionOutput) OVER (PARTITION BY StateABB) AS MaxEmissionInState,
        PlantName, 
        UtilityName,
        ROW_NUMBER() OVER (PARTITION BY StateABB ORDER BY EmissionOutput DESC) AS RowNum
    FROM 
        Hackathon_LH.dbo_UnadjustedEmissions
    WHERE 
        EmissionOutput > (SELECT AVG(EmissionIutput) FROM Hackathon_LH.dbo_UnadjustedEmissions)
             AND YEAR = 2022
   AND StateABB IN ('TN', 'AR', 'MS')
)
SELECT 
    FacilityCode, 
    StateABB, 
    EmissionOutput AS MaxEmissionOutput, 
    PlantName, 
    UtilityName
FROM 
    MaxEmissionPerState
WHERE 
    RowNum = 1
ORDER BY 
    MaxEmissionInState DESC, StateABB, EmissionOutput DESC;


StatementMeta(, c561cafb-184d-4bae-8a49-40e6b08b4895, 44, Finished, Available)

<Spark SQL result set with 3 rows and 5 fields>

In [60]:
-- This query returns a list of power plants with different type emissions and number Generator's. 
SELECT PlantName,
    MAX(UnadjustedSO2) AS UnadjustedSO2_Max,
    MAX(UnadjustedCO2) AS UnadjustedCO2_Max,
    MAX(UnadjustedHeat) AS UnadjustedHeat_Max,
    MAX(UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_Max,
    COUNT(DISTINCT GeneratorID) AS NumberOfGenerator,
     COUNT(DISTINCT GenFuelType) AS GeneratorFuelType,
    GenRetirementYear
FROM Hackathon_LH.dbo_UnadjustedEmissions
WHERE EmissionOutput > (SELECT AVG(EmissionIutput) FROM Hackathon_LH.dbo_UnadjustedEmissions)
-- Adding NULL for a conditional statement to include the list all of generators with no expiration date ## When LIMIT is more that 5rows
AND (GenRetirementYear > 2024 OR GenRetirementYear is null) 
GROUP BY PlantName, GenRetirementYear
ORDER BY GenRetirementYear DESC
LIMIT 5

StatementMeta(, c561cafb-184d-4bae-8a49-40e6b08b4895, 61, Finished, Available)

<Spark SQL result set with 5 rows and 8 fields>

## List of power plants that their emission output is not equal to the input as require by law.
##### Read more online at https://www.epa.gov/renewable-fuel-standard-program/overview-renewable-fuel-standard

In [36]:
--This query returns the list of power plants that generate more NOx emissions above 100 lb/MWh.
-- And theur emissions output is above 50% compare to their electricity convertion rate is less the 50%.
-- Along with the 50%, 60%, 80%, and 100% percentile's that can be used as a KPI tracking mechanism for engaging in activies that will help reduce the pollution rate.

WITH AllData AS (
    SELECT
    PlantName,
    StateABB,
      MAX(UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_Max,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_50th,
    PERCENTILE_CONT(0.6) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_60th,
    PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_80th,
    PERCENTILE_CONT(1.0) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_100th
FROM
    Hackathon_LH.dbo_UnadjustedEmissions
WHERE 
    EmissionOutput > (SELECT AVG(EmissionOutput) FROM Hackathon_LH.dbo_UnadjustedEmissions)
 GROUP BY
    PlantName , StateABB
ORDER BY
    UnadjustedNOxEmissions_50th DESC)

   --- SELECT * FROM AllData WHERE UnadjustedNOxEmissions_50th != UnadjustedNOxEmissions_60th
     SELECT PlantName, UnadjustedNOxEmissions_50th, UnadjustedNOxEmissions_60th, UnadjustedNOxEmissions_80th, UnadjustedNOxEmissions_100th, StateABB
     FROM AllData
     WHERE UnadjustedNOxEmissions_100th > UnadjustedNOxEmissions_60th
     AND UnadjustedNOxEmissions_50th > 100
       

StatementMeta(, 6f2622ce-b6e3-440d-937c-aee85fe73517, 37, Finished, Available)

<Spark SQL result set with 4 rows and 6 fields>

# Switching Section to R Programming Language 

In [ ]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("MyApp").getOrCreate()

# Read data from the specified path into a DataFrame (as Delta Parquet)
result_df = spark.read.format("delta").load("abfss://c03564d3-6b73-4fbb-a507-4a1f6446db7c@onelake.dfs.fabric.microsoft.com/c18dcf4f-975e-42ff-8f98-b7bac72a6e92/Tables/dbo_UnadjustedEmissions")

# Show the first few rows of the DataFrame
result_df.show(10)



In [138]:
# Load the SparkR package
library(SparkR)
spark <- sparkR.session(appName = "MySparkApp")



# Read data from the specified path into a SparkDataFrame (as Delta Parquet)
df <- read.df("abfss://c03564d3-6b73-4fbb-a507-4a1f6446db7c@onelake.dfs.fabric.microsoft.com/c18dcf4f-975e-42ff-8f98-b7bac72a6e92/Tables/dbo_UnadjustedEmissions", source = "delta")
head(df)  

StatementMeta(, fe781bfa-1b97-4495-9ed2-5f73db71e60e, 140, Finished, Available)

 [1] YEAR                        FacilityCode               
 [3] PlantName                   StateABB                   
 [5] UnitOperation               FuelType                   
 [7] UnadjustedHeat              UnadjustedOzoneFrmHeat     
 [9] UnadjustedNOxEmissions      UnadjustedOzoneNOxEmissions
[11] UnadjustedSO2               UnadjustedCO2              
[13] UnadjustedHg                GeneratorID                
[15] GenFuelType                 GenMoverType               
[17] GenMWH                      GenOzoneMWH                
[19] GenYearOnline               GenRetirementYear          
[21] UtilityName                 EmissionOutput             
[23] EmissionIutput             
<0 rows> (or 0-length row.names)
Warning message:
In sparkR.session(appName = "MySparkApp") :
  Version mismatch between Spark JVM and SparkR package. JVM version was 3.4.1.5.3-117503204, while R package version was 3.4.1

In [139]:
head(df, n = 5)


StatementMeta(, fe781bfa-1b97-4495-9ed2-5f73db71e60e, 141, Finished, Available)

  YEAR FacilityCode PlantName StateABB UnitOperation FuelType UnadjustedHeat
1 2022         3407  Kingston       TN            OP      SUB        3475443
2 2022         3407  Kingston       TN            OP      SUB        3475443
3 2022         3407  Kingston       TN            OP      SUB        3475443
4 2022         3407  Kingston       TN            OP      SUB        3475443
5 2022         3407  Kingston       TN            OP      SUB        3475443
  UnadjustedOzoneFrmHeat UnadjustedNOxEmissions UnadjustedOzoneNOxEmissions
1                1595013                142.473                      59.465
2                1595013                142.473                      59.465
3                1595013                142.473                      59.465
4                1595013                142.473                      59.465
5                1595013                142.473                      59.465
  UnadjustedSO2 UnadjustedCO2 UnadjustedHg GeneratorID GenFuelType GenMoverType
1 

In [166]:
# Assuming 'df' is your SparkDataFrame
selected_df <- df[, c("GeneratorID", "PlantName", "UnadjustedNOxEmissions")]
showDF(selected_df)


StatementMeta(, fe781bfa-1b97-4495-9ed2-5f73db71e60e, 168, Finished, Available)

+-----------+---------+----------------------+
|GeneratorID|PlantName|UnadjustedNOxEmissions|
+-----------+---------+----------------------+
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2271.059|
|          2|   Cayuga|              2001.128|
|          2|   Cayuga|              2001.128|
|          2|

In [31]:
selected_df <- select(df, "GeneratorID", "PlantName", "UnadjustedNOxEmissions", "UnadjustedOzoneNOxEmissions")
filtered_df <- filter(selected_df, selected_df$UnadjustedNOxEmissions > 0)
summary_df <- summarize(filtered_df, avg_NOx = avg(filtered_df$UnadjustedNOxEmissions))

showDF(summary_df)


StatementMeta(, fe781bfa-1b97-4495-9ed2-5f73db71e60e, 33, Finished, Available)

+-----------------+
|          avg_NOx|
+-----------------+
|67.06073237124117|
+-----------------+

In [37]:
selected_df <- select(df, "GeneratorID", "PlantName", "UnadjustedNOxEmissions", "UnadjustedOzoneNOxEmissions")
filtered_df <- filter(selected_df, selected_df$UnadjustedNOxEmissions > 0)
summary_df <- summarize(filtered_df, Numof_Gen = count(filtered_df$GeneratorID))

showDF(summary_df)


StatementMeta(, fe781bfa-1b97-4495-9ed2-5f73db71e60e, 39, Finished, Available)

+---------+
|Numof_Gen|
+---------+
|  1384752|
+---------+

In [7]:
# Read data with R from the specified path into a DataFrame (as Delta Parquet)
df <- read.df("abfss://c03564d3-6b73-4fbb-a507-4a1f6446db7c@onelake.dfs.fabric.microsoft.com/c18dcf4f-975e-42ff-8f98-b7bac72a6e92/Tables/dbo_UnadjustedEmissions", source = "delta")

# Register the DataFrame as a temporary view
createOrReplaceTempView(df, "AllDataResult")

# Execute your SQL query
result_df <- sql("WITH AllData AS (
    SELECT
        PlantName,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_50th,
        PERCENTILE_CONT(0.6) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_60th,
        PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_80th,
        PERCENTILE_CONT(1.0) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_100th
    FROM
        AllDataResult
    WHERE
        EmissionOutput > (SELECT AVG(EmissionOutput) FROM AllDataResult)
    GROUP BY
        PlantName
    ORDER BY
        UnadjustedNOxEmissions_50th DESC
)
SELECT * FROM AllData WHERE UnadjustedNOxEmissions_100th > UnadjustedNOxEmissions_80th")

# Show the result
result_df_local <- collect(result_df)
showDF(result_df)


StatementMeta(, 37f1191e-0895-4296-bdc0-6896b5bbbb7a, 9, Finished, Available)

+--------------------+---------------------------+---------------------------+---------------------------+----------------------------+
|           PlantName|UnadjustedNOxEmissions_50th|UnadjustedNOxEmissions_60th|UnadjustedNOxEmissions_80th|UnadjustedNOxEmissions_100th|
+--------------------+---------------------------+---------------------------+---------------------------+----------------------------+
|      Princeton (IL)|                    283.837|                    320.226|                    509.451|                     640.452|
|          Heber City|                      41.52|                      41.52|                     50.262|                      54.632|
|         Caterpillar|         34.726499999999994|                     68.201|                     85.251|                     102.301|
|Northeast Generat...|                     32.087|                      35.19|                     40.781|                      41.526|
|                 Tok|                    31.873

In [18]:
 
# Load the necessary library for plotting
library(ggplot2)



# Create a bar chart
ggplot(result_df_local , aes(x = PlantName, y = UnadjustedNOxEmissions_50th)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  labs(title = "50th Percentile NOx Emissions by Plant",
       x = "Plant Name",
       y = "NOx Emissions")

StatementMeta(, 37f1191e-0895-4296-bdc0-6896b5bbbb7a, 20, Finished, Available)

In [38]:
# Load the necessary libraries
library(ggplot2)
library(SparkR)


# Read data with R from the specified path into a DataFrame (as Delta Parquet)
df <- read.df("abfss://c03564d3-6b73-4fbb-a507-4a1f6446db7c@onelake.dfs.fabric.microsoft.com/c18dcf4f-975e-42ff-8f98-b7bac72a6e92/Tables/dbo_UnadjustedEmissions", source = "delta")

# Register the DataFrame as a temporary view
createOrReplaceTempView(df, "AllDataResult")

# Execute your SQL query
result_df <- sql("WITH AllData AS (
    SELECT
        PlantName,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_50th,
        PERCENTILE_CONT(0.6) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_60th,
        PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_80th,
        PERCENTILE_CONT(1.0) WITHIN GROUP (ORDER BY UnadjustedNOxEmissions) AS UnadjustedNOxEmissions_100th
    FROM
        AllDataResult
    WHERE
        EmissionOutput > (SELECT AVG(EmissionOutput) FROM AllDataResult)
    GROUP BY
        PlantName
    ORDER BY
        UnadjustedNOxEmissions_50th DESC
)
SELECT * FROM AllData WHERE UnadjustedNOxEmissions_100th > UnadjustedNOxEmissions_80th")
 
result_df_local <- collect(result_df)

# Create a scatter plot
ggplot(result_df_local, aes(x = UnadjustedNOxEmissions_50th, y = UnadjustedNOxEmissions_60th)) +
  geom_point(color = "blue", size = 3) +
  labs(title = "Scatter Plot: 50th vs. 60th Percentile NOx Emissions",
       x = "50th Percentile NOx Emissions",
       y = "60th Percentile NOx Emissions")


StatementMeta(, 75b8f037-267d-4c48-842e-4727da7f5321, 39, Finished, Available)

In [41]:

# Create a bar chart
p <- ggplot(result_df_local, aes(x = PlantName, y = UnadjustedNOxEmissions_50th)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  labs(title = "50th Percentile NOx Emissions by Plant",
       x = "Plant Name",
       y = "NOx Emissions")

# Print the plot
print(p)


StatementMeta(, 75b8f037-267d-4c48-842e-4727da7f5321, 42, Finished, Available)

In [73]:
# Load the necessary libraries
library(ggplot2)
library(SparkR)
 
result_df_local <- collect(result_df)

barplot(result_df_local$UnadjustedNOxEmissions_50th, names.arg = result_df_local$PlantName, col = "skyblue",
        main = "Bar Chart: 50th Percentile NOx Emissions by Plant",
        xlab = "Plant Name", ylab = "NOx Emissions")

StatementMeta(, 75b8f037-267d-4c48-842e-4727da7f5321, 74, Finished, Available)